会津若松市の公用車走行のOpenDataと、OSMの信号機の場所の2つのデータを加工し、急停止/急加速した場所をGoogle Mapにプロットしてみる。

In [109]:
from math import sqrt, cos, sin, asin, radians, atan2
import json
import requests
import gmaps
import numpy as np

まず、公用車の走行データをData4Citizienより取得。1回の上限が1万件なので、offsetをつかって10回に分けてデータを取得する(10万件)。データは、100ms周期の3軸加速度が取得できる。

In [4]:
def getCarDatasets():
  LIMIT = 10000
  SIZE = 100000
  
  for i in xrange(SIZE/LIMIT):
    pos_offset = LIMIT * (i)
    url = 'http://www.data4citizen.jp/app/developer/query/query?odql=SELECT%20car_name%0A%2C%20latitude%0A%2C%20longitude%0A%2C%20gps_error_meter%0A%2C%20accel_x_transverse%0A%2C%20accel_y_longitudinal%0A%2C%20accel_z_vertical%0AFROM%20O_CAR_TRAFFIC_DATA%20limit%20'+str(LIMIT)+'%20offset%20'+str(pos_offset)
    r = requests.get(url)
    if i == 0:
      data = r.json()["data"]
    else:
      data.extend(r.json()["data"])

  return data

会津若松市の公用車データ10万件のデータを取得。

In [5]:
carDatasets = getCarDatasets()
len(carDatasets)

100000

次に、OSMから、信号機の緯度/経度を取得。

In [75]:
def getTrafficLights():
  url = 'https://gist.githubusercontent.com/anonymous/1ebabeb9622ae01338f33e1f364975fd/raw/9018ebf7046227d6e911c6eab6aee88054fb87ce/overpass.geojson'
  r = requests.get(url)
  data = r.json()
  lightPoints = np.empty((0,2), float)
  for item in data["features"]:
    lon = item["geometry"]["coordinates"][0]
    lat = item["geometry"]["coordinates"][1]
    lightPoints = np.append(lightPoints, np.array([[lat, lon]]), axis=0)
    
  return lightPoints

信号機データを取得。

In [88]:
lightPoints = getTrafficLights()
len(lightPoints)

260

In [84]:
def accident(before_axis, latest_axis):
  ACCIDENT = 10000
  if np.power((before_axis*100-latest_axis*100),2) > ACCIDENT:
    return True
  else:
    return False
  
accidentPoints = np.empty((0,2), float)
(x, y, z) = (0, 0, 0)
for item in carDatasets:
  lat = item["latitude"]
  lon = item["longitude"]
  (_x, _y, _z) = (x, y, z)
  x = item["accel_x_transverse"]
  y = item["accel_y_longitudinal"]
  z = item["accel_z_vertical"]
  ACCIDENT = 5000
  if accident(_x, x) or accident(_y, y) or accident(_z, z):
    accidentPoints = np.append(accidentPoints, np.array([[lat, lon]]), axis=0)

len(accidentPoints)

1227

次に、信号機の近隣かどうかの判定をおこなう

In [118]:
def nearTfafficLights(lon1, lat1, lon2, lat2):
  radius = 6371 * 1000
  dlat = radians(lat2-lat1)
  dlon = radians(lon2-lon1)
  a = sin(dlat/2) * sin(dlat/2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2) * sin(dlon/2)
  c = 2 * atan2(sqrt(a), sqrt(1-a))
  d = radius * c

  return d

realAccidentPoints = np.empty((0,2), float)
for apoint in accidentPoints:
  for lpoint in lightPoints:
    distance = nearTfafficLights(apoint[0], apoint[1], lpoint[0], lpoint[1])
    if distance > 10:
      realAccidentPoints = np.append(realAccidentPoints, np.array([[apoint[0], apoint[1]]]), axis=0)
      break

print realAccidentPoints

[[  37.494257  139.929316]
 [  37.497904  139.930645]
 [  37.497913  139.930651]
 ..., 
 [  37.505968  139.931962]
 [  37.513885  139.931663]
 [  37.551505  139.92517 ]]


最後にGoogle Map上にPlot。

In [119]:
gmaps.configure(api_key="AIza.....")
    
m = gmaps.Map()
m.add_layer(gmaps.Heatmap(data=realAccidentPoints.tolist()))
m